# Setup and Installation

In this step, we ensure that we have DuckDB and the spatial extension installed and accessible.


In [2]:
%pip install duckdb
# If a spatial extension or duckdb-spatial is available, install that as well.
# The syntax might vary depending on the duckdb version.


Note: you may need to restart the kernel to use updated packages.


# Load and Configure DuckDB with Spatial Extensions

Here, we will:
- Create a DuckDB in-memory database connection
- Load the spatial extensions


In [6]:
import duckdb

# Create an in-memory DuckDB connection
con = duckdb.connect()

# Install and load the spatial extension if needed
con.execute("INSTALL spatial")
con.execute("LOAD spatial")

# Check that the extension is working by listing available functions
con.execute("SELECT * FROM duckdb_functions() WHERE function_name LIKE 'ST_%' LIMIT 5").fetchdf()
# con.execute("SELECT * FROM duckdb_functions() ").fetchdf()

,database_name,database_oid,schema_name,function_name,function_type,description,comment,tags,return_type,parameters,parameter_types,varargs,macro_definition,has_side_effects,internal,function_oid,example,stability
0,system,0,main,ST_Read_Meta,table,Read the metadata from a variety of geospatial...,None,{'ext': 'spatial'},None,[col0],[VARCHAR],None,None,NaN,True,2028,-- Find the coordinate reference system author...,None
1,system,0,main,ST_Read_Meta,table,Read the metadata from a variety of geospatial...,None,{'ext': 'spatial'},None,[col0],[VARCHAR[]],None,None,NaN,True,2028,-- Find the coordinate reference system author...,None
2,system,0,main,ST_ReadSHP,table,None,None,{},None,"[col0, encoding]","[VARCHAR, VARCHAR]",None,None,NaN,True,1999,None,None
3,system,0,main,ST_ReadOSM,table,The `ST_ReadOsm()` table function enables read...,None,{'ext': 'spatial'},None,[col0],[VARCHAR],None,None,NaN,True,1997,SELECT *\nFROM ST_ReadOSM('tmp/data/germany.os...,None
4,system,0,main,ST_Read,table,Read and import a variety of geospatial file f...,None,{'ext': 'spatial'},None,"[col0, keep_wkb, max_batch_size, sequential_la...","[VARCHAR, BOOLEAN, INTEGER, BOOLEAN, VARCHAR, ...",None,None,NaN,True,2022,-- Read a Shapefile\nSELECT * FROM ST_Read('so...,None


# Generate Synthetic Data

We will create small synthetic datasets:
- `states`: 5 polygons representing different states.
- `rivers`: 6 lines representing rivers crossing multiple states.
- `cities`: 10 points representing cities scattered around these polygons.

For simplicity, let's assume coordinates are in a planar coordinate reference system that uses meters. The geometry will be small scale polygons and lines in a random but somewhat structured layout.


In [7]:
import pandas as pd

# Synthetic states: We'll define simple polygons as squares or rectangles.
# We'll place them roughly in a grid pattern.

states_data = [
    (1, "State A", "POLYGON((0 0, 0 100000, 100000 100000, 100000 0, 0 0))"),
    (2, "State B", "POLYGON((100000 0, 100000 100000, 200000 100000, 200000 0, 100000 0))"),
    (3, "State C", "POLYGON((0 100000, 0 200000, 100000 200000, 100000 100000, 0 100000))"),
    (4, "State D", "POLYGON((100000 100000, 100000 200000, 200000 200000, 200000 100000, 100000 100000))"),
    (5, "State E", "POLYGON((200000 0, 200000 100000, 300000 100000, 300000 0, 200000 0))")
]

# Synthetic rivers: lines that cross multiple states.
# We'll define lines that run horizontally and vertically across the state grid.
# For diversity, some rivers cross state boundaries:
rivers_data = [
    (1, "River Alpha", "LINESTRING(50000 0, 50000 200000)"),      # Vertical line cutting through State A and C
    (2, "River Beta",  "LINESTRING(150000 0, 150000 200000)"),   # Vertical line cutting through B, D
    (3, "River Gamma", "LINESTRING(250000 0, 250000 100000)"),   # Vertical line cutting through E
    (4, "River Delta", "LINESTRING(0 50000, 300000 50000)"),     # Horizontal line crossing A, B, E
    (5, "River Epsilon","LINESTRING(0 150000, 300000 150000)"),  # Horizontal line crossing C, D
    (6, "River Zeta",   "LINESTRING(30000 30000, 130000 170000)")# Diagonal line crossing multiple states
]

# Synthetic cities: points scattered around.
# We'll place some inside states and near rivers.
cities_data = [
    (1, "City 1", "POINT(50000 50000)"),    # In State A, near River Alpha and Delta
    (2, "City 2", "POINT(150000 50000)"),   # In State B, near River Beta and Delta
    (3, "City 3", "POINT(250000 50000)"),   # In State E, near River Gamma and Delta
    (4, "City 4", "POINT(50000 150000)"),   # In State C, near River Alpha and Epsilon
    (5, "City 5", "POINT(150000 150000)"),  # In State D, near River Beta and Epsilon
    (6, "City 6", "POINT(250000 150000)"),  # Just in State E (actually out of range?), near Epsilon but far from Gamma?
    (7, "City 7", "POINT(70000 70000)"),    # In State A, somewhat near Zeta
    (8, "City 8", "POINT(120000 120000)"),  # Intersection area B/D boundary, near Beta, Epsilon, Zeta
    (9, "City 9", "POINT(280000 50000)"),   # In State E, near Gamma & Delta
    (10,"City 10","POINT(40000 140000)")    # In State C, near Alpha and Epsilon, possibly near Zeta
]

states_df = pd.DataFrame(states_data, columns=["state_id", "state_name", "wkt"]) #wkt: well-known text
rivers_df = pd.DataFrame(rivers_data, columns=["river_id", "river_name", "wkt"])
cities_df = pd.DataFrame(cities_data, columns=["city_id", "city_name", "wkt"])

states_df


,state_id,state_name,wkt
0,1,State A,"POLYGON((0 0, 0 100000, 100000 100000, 100000 ..."
1,2,State B,"POLYGON((100000 0, 100000 100000, 200000 10000..."
2,3,State C,"POLYGON((0 100000, 0 200000, 100000 200000, 10..."
3,4,State D,"POLYGON((100000 100000, 100000 200000, 200000 ..."
4,5,State E,"POLYGON((200000 0, 200000 100000, 300000 10000..."


# Insert Data into DuckDB Tables

# todo:  convert the WKT strings into actual `GEOMETRY` objects.


In [26]:
con.execute("DROP TABLE IF EXISTS states;")
con.execute("DROP TABLE IF EXISTS rivers;")
con.execute("DROP TABLE IF EXISTS cities;")

con.execute("""
CREATE TABLE states (state_id INT, state_name TEXT, geom GEOMETRY);
""")
con.execute("""
CREATE TABLE rivers (river_id INT, river_name TEXT, geom GEOMETRY);
""")
con.execute("""
CREATE TABLE cities (city_id INT, city_name TEXT, geom GEOMETRY);
""")
print("DataFrame wkt column type:", states_df['wkt'].dtype)
states_df['wkt'] = states_df['wkt'].astype(str)
print("DataFrame wkt column type:", states_df['wkt'].dtype)

# Insert data using prepared statements
for row in states_df.itertuples(index=False):
    states_df['wkt'] = states_df['wkt'].astype(str)
    #print("DataFrame wkt column type:3", str(states_df['wkt']))
    con.execute(
        "INSERT INTO states VALUES (?, ?, ?)",
        [row.state_id, row.state_name, row.wkt]
    )

for row in rivers_df.itertuples(index=False):
    con.execute(
        "INSERT INTO rivers VALUES (?, ?, ?)",
        [row.river_id, row.river_name, row.wkt]
    )

for row in cities_df.itertuples(index=False):
    con.execute(
        "INSERT INTO cities VALUES (?, ?, ?)",
        [row.city_id, row.city_name, row.wkt]
    )

# Check that data was inserted
con.execute("SELECT * FROM states LIMIT 5").df()


DataFrame wkt column type: object
DataFrame wkt column type: object


,state_id,state_name,geom
0,1,State A,"[2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,State B,"[2, 4, 0, 0, 0, 0, 0, 0, 0, 80, 195, 71, 0, 0,..."
2,3,State C,"[2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 80, 19..."
3,4,State D,"[2, 4, 0, 0, 0, 0, 0, 0, 0, 80, 195, 71, 0, 80..."
4,5,State E,"[2, 4, 0, 0, 0, 0, 0, 0, 0, 80, 67, 72, 0, 0, ..."


In [33]:
# If supported by your DuckDB version
# If not needed, skip this step.
try:
    con.execute("CREATE INDEX idx_states_geom ON states USING GIST (geom);")
    con.execute("CREATE INDEX idx_rivers_geom ON rivers USING GIST (geom);")
    con.execute("CREATE INDEX idx_cities_geom ON cities USING GIST (geom);")
except:
    # If indexing is not supported, just continue
    pass


# Find Rivers Intersecting Each State




In [34]:
con.execute("""
CREATE OR REPLACE VIEW state_river_counts AS
SELECT s.state_id,
       s.state_name,
       COUNT(DISTINCT r.river_id) AS num_rivers_intersecting
FROM states s
JOIN rivers r ON ST_Intersects(s.geom, r.geom)
GROUP BY s.state_id, s.state_name;
""")

con.execute("SELECT * FROM state_river_counts;").df()


,state_id,state_name,num_rivers_intersecting
0,5,State E,2
1,4,State D,3
2,2,State B,2
3,1,State A,3
4,3,State C,3


# Determine Cities Within Each State and Within 50 km of a River

- First, find which cities are contained by each state using `ST_Contains()`.
- Then, find which cities are within 50 km (50000m) of a river using `ST_DWithin()`.
- Finally, combine these results.


In [37]:
con.execute("""
CREATE OR REPLACE VIEW city_state_map AS
SELECT s.state_id,
       s.state_name,
       c.city_id,
       c.city_name
FROM states s
JOIN cities c ON ST_Contains(s.geom, c.geom)
""")

con.execute("SELECT * FROM city_state_map;").df()

,state_id,state_name,city_id,city_name
0,1,State A,1,City 1
1,1,State A,7,City 7
2,2,State B,2,City 2
3,3,State C,4,City 4
4,3,State C,10,City 10
5,4,State D,5,City 5
6,4,State D,8,City 8
7,5,State E,3,City 3
8,5,State E,9,City 9


Now, we identify which cities are within 50 km of at least one river.


In [39]:
con.execute("""
CREATE OR REPLACE VIEW city_river_buffer AS
SELECT 
       c.city_id,
       c.city_name,
       r.river_id,
       r.river_name
FROM  cities c
JOIN rivers r ON ST_DWithin(c.geom, r.geom, 50000)
""")

con.execute("SELECT * FROM city_state_map;").df()

,state_id,state_name,city_id,city_name
0,1,State A,1,City 1
1,1,State A,7,City 7
2,2,State B,2,City 2
3,3,State C,4,City 4
4,3,State C,10,City 10
5,4,State D,5,City 5
6,4,State D,8,City 8
7,5,State E,3,City 3
8,5,State E,9,City 9



Now we combine these to get cities that are in a state and also within 50 km of a river.


In [42]:
con.execute("""
CREATE OR REPLACE VIEW state_city_river_buffer AS
SELECT 
       cs.state_id,
       cs.state_name,
       cs.city_id,
       cs.city_name
FROM  city_state_map cs
WHERE cs.city_id IN (SELECT city_id FROM city_river_buffer)
""")

con.execute("SELECT * FROM state_city_river_buffer;").df()

,state_id,state_name,city_id,city_name
0,1,State A,1,City 1
1,1,State A,7,City 7
2,2,State B,2,City 2
3,3,State C,4,City 4
4,3,State C,10,City 10
5,4,State D,5,City 5
6,4,State D,8,City 8
7,5,State E,3,City 3
8,5,State E,9,City 9
